NOTES

1. Serve the model using vLLM like so - vllm serve Qwen/Qwen3-30B-A3B-Instruct-2507 --tensor-parallel-size 4 --port 8000 
2. 

In [1]:
import os
from pydantic import BaseModel
import guidance

In [2]:
# from pydantic import BaseModel, Field, conint
# from typing import Literal

# class Person(BaseModel):
#     name: str
#     age: conint(ge=0, le=150)
#     country: Literal["UK", "IN"]

# schema: dict = Person.model_json_schema()   # Pydantic v2
# # schema_v1: dict = Person.schema()        # Pydantic v1
# schema

In [3]:
# # vllm hosted model using LiteLLM

# litellm_desc = {
#     "model_name": "Qwen/Qwen3-30B-A3B-Instruct-2507",
#     "litellm_params": {  # params for litellm completion/embedding call
#         "model": "hosted_vllm/Qwen/Qwen3-30B-A3B-Instruct-2507",
#         "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
#         "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
#     },
# }
# base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=True)

In [2]:
# using sglang hosted model
# need to add the experimental sglang from nightly version of guidance - https://github.com/guidance-ai/guidance/blob/main/guidance/models/experimental/_sglang.py

base_lm = guidance.models.experimental.SglangModel(model="Qwen/Qwen3-4B-Instruct-2507", base_url="http://127.0.0.1:40000/v1" ,echo=True, api_key = os.environ.get("OPENAI_API_KEY", "NO_KEY"))

In [3]:
def run_gen_test(lm):
    with guidance.user():
        lm += "What is the capital of France? and its population?"
        lm += "Format your answer as follows: Capital: <capital>, Population: <population>"

    with guidance.assistant():
        lm += guidance.gen(max_tokens=1024, temperature=0.7, name="answer")
        print(lm["answer"])

run_gen_test(base_lm)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Capital: Paris, Population: 2,161,000


In [5]:
# %%bash

# module purge
# module load gcc-uoneasy/12.3.0
# module load CUDA/12.4.0
# export CC=gcc
# export CXX=g++
# # Remove TRITON_BUILD_WITH_CLANG_LLD=1 as it's Clang-specific; GCC may not need it
# rm -rf ~/.triton ~/.cache/triton
# export VLLM_WORKER_MULTIPROC_METHOD=spawn
# module load anaconda-uoneasy/2023.09-0
# source /gpfs01/home/ppxac9/.zshrc
# conda activate envname

In [4]:
import torch
torch.cuda.empty_cache()

import time
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union, Literal, ForwardRef, List, Any
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json, gen
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json
import guidance
from utils import timing_decorator

In [5]:

# with open("../output/2025-airport-charges-terms-and-conditions/tinychargesmarkdown.md", "r") as f:
#     markdown_content = f.read()
# with open("../output/2025-airport-charges-terms-and-conditions/2025-airport-charges-terms-and-conditions.md", "r") as f:
#     markdown_content = f.read()    


In [6]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3,4,5,6"

In [7]:

# # MODEL_ID = "Qwen/Qwen3-30B-A3B"
# MODEL_ID = "Qwen/Qwen3-30B-A3B-Instruct-2507"
# hf_model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     device_map="auto",       
#     low_cpu_mem_usage=True,          
# )


# tok        = AutoTokenizer.from_pretrained(MODEL_ID)


In [8]:
# from guidance.chat import ChatTemplate
# class QwenChatTemplate(ChatTemplate):
#     template_str = tok.chat_template
#     def get_role_start(self, role_name):
#         # adjust tokens if you use a derivative model with different tags
#         if role_name == "system":
#             return "<|im_start|>system\n"
#         elif role_name == "user":
#             return "<|im_start|>user\n"
#         elif role_name == "assistant":
#             return "<|im_start|>assistant\n"
#         else:
#             raise UnsupportedRoleException(role_name, self)

#     def get_role_end(self, role_name=None):
#         # same token for every role in Qwen‑3
#         return "<|im_end|>\n"

In [9]:

# model = guidance.models.Transformers(hf_model, tok, chat_template=QwenChatTemplate, echo=False)

In [10]:
from pydantic import BaseModel, Field
from typing import List, Any

class DomainVariable(BaseModel):
    """Defines a single variable the LLM can use in the computation graph."""
    name: str = Field(..., description="The unique identifier for the variable.")
    description: str = Field(..., description="A detailed explanation of what this variable represents.")
    # Optional: You could add type hints, units, etc. for more advanced validation
    unit: Optional[str] = Field(..., description="The unit of the variable")
    data_type : type = Field(..., description="The data type of the variable")


In [11]:
# ALL_VARIABLES = {
    
#     'transfer_passenger_count': DomainVariable(name='transfer_passenger_count', description='Total number of transferring passengers.', unit=None, data_type=float),
#     'airline_scheduling_season': DomainVariable(name='airline_scheduling_season', description='Whether summer/winter airline scheduling season.', unit=None, data_type=float),
#     'takeoff_aircraft_mtow_tonnes': DomainVariable(name='takeoff_aircraft_mtow_tonnes', description='The Maximum Take-Off Weight in tonnes.', unit='tonne', data_type=float),
#     'landing_aircraft_mtow_tonnes': DomainVariable(name='landing_aircraft_mtow_tonnes', description='The Maximum Landing Weight in tonnes.', unit='tonne', data_type=float),
    
#     'parking_duration_hours': DomainVariable(
#         name='parking_duration_hours',
#         description='Total duration of parking in hours. Surcharges apply at 48 and 72 hours.',
#         unit='hours',
#         data_type=float
#     ),
#     'aircraft_stand_type': DomainVariable(
#         name='aircraft_stand_type',
#         description='The type of aircraft stand used for parking. E.g., "Wide Contact", "Narrow Remote", "LAP", "Long Term Remote".',
#         unit=None,
#         data_type=str
#     ),
#     'parking_location': DomainVariable(
#         name='parking_location',
#         description='The location of the parking stand, either "EAP" (East Aerodrome Parking) or "WAP" (West Aerodrome Parking).',
#         unit=None,
#         data_type=str
#     ),
#     'is_overnight_parking': DomainVariable(
#         name='is_overnight_parking',
#         description='True if the parking occurs during the free overnight period (2300-0600hrs).',
#         unit=None,
#         data_type=bool
#     )
# }
# # 2. Map charge types to the variable names they are allowed to use
# CHARGE_CATEGORY_VARIABLES = {
#     # --- Existing Categories ---
#     "transfer_passenger_charge": [
#         'transfer_passenger_count', 
#         'airline_scheduling_season'
#     ],
#     "runway_landing_charge": [
#         'landing_aircraft_mtow_tonnes', 
#         'airline_scheduling_season'
#     ],
#     "runway_takeoff_charge": [
#         'takeoff_aircraft_mtow_tonnes', 
#         'airline_scheduling_season'
#     ],

#     # --- New Categories for Parking Charges ---
#     "east_aerodrome_parking_charge": [
#         'parking_duration_hours',
#         'aircraft_stand_type',
#         'is_overnight_parking'
#         # 'parking_location' is implicitly 'EAP' for this category
#     ],
#     "west_aerodrome_parking_charge": [
#         'parking_duration_hours',
#         'aircraft_stand_type',
#         'is_overnight_parking'
#         # 'parking_location' is implicitly 'WAP' for this category
#     ]
# }



In [12]:
def create_dynamic_variable_enum(charge_category: str, charge_category_variables: dict) -> type(Enum):
    """
    Creates a new Enum class containing only the variables relevant
    to the specified charge category.
    """
    variable_names = charge_category_variables.get(charge_category)
    if not variable_names:
        raise ValueError(f"Unknown charge category: {charge_category}")
    
    # The dictionary for the Enum must have {MEMBER_NAME: value}
    # We'll use uppercase for the member name for convention.
    enum_dict = {name.upper(): name for name in variable_names}
    
    # Create the Enum class dynamically
    return Enum("Var", enum_dict)

# Var = create_dynamic_variable_enum("transfer_passenger_charge", CHARGE_CATEGORY_VARIABLES)
# print(Var.TRANSFER_PASSENGER_COUNT.value)

In [13]:
# allowed_variables = [el.value for el in list(Var)]
# allowed_variables

In [14]:
from enum import Enum

class Units(str, Enum):
    # General
    UNITLESS = "UNITLESS"
    PERCENT = "PERCENT"

    # Time
    NANOSECONDS = "NANOSECONDS"
    MICROSECONDS = "MICROSECONDS"
    MILLISECONDS = "MILLISECONDS"
    SECONDS = "SECONDS"
    MINUTES = "MINUTES"
    HOURS = "HOURS"
    DAYS = "DAYS"
    WEEKS = "WEEKS"

    # Length
    MILLIMETERS = "MILLIMETERS"
    CENTIMETERS = "CENTIMETERS"
    METERS = "METERS"
    KILOMETERS = "KILOMETERS"
    INCHES = "INCHES"
    FEET = "FEET"
    YARDS = "YARDS"
    MILES = "MILES"

    # Mass
    MILLIGRAMS = "MILLIGRAMS"
    GRAMS = "GRAMS"
    KILOGRAMS = "KILOGRAMS"
    TONNES = "TONNES"

    # Speed / Acceleration
    METERS_PER_SECOND = "METERS_PER_SECOND"
    KILOMETERS_PER_HOUR = "KILOMETERS_PER_HOUR"
    MILES_PER_HOUR = "MILES_PER_HOUR"
    METERS_PER_SECOND_SQUARED = "METERS_PER_SECOND_SQUARED"

    # Data size
    BYTES = "BYTES"
    KILOBYTES = "KILOBYTES"
    MEGABYTES = "MEGABYTES"
    GIGABYTES = "GIGABYTES"
    TERABYTES = "TERABYTES"
    BITS = "BITS"

    # Data rates
    BITS_PER_SECOND = "BITS_PER_SECOND"
    KILOBITS_PER_SECOND = "KILOBITS_PER_SECOND"
    MEGABITS_PER_SECOND = "MEGABITS_PER_SECOND"
    GIGABITS_PER_SECOND = "GIGABITS_PER_SECOND"
    BYTES_PER_SECOND = "BYTES_PER_SECOND"

    # Currencies (suggest using ISO codes)
    EUROS = "EUROS"
    USD = "USD"
    GBP = "GBP"
    JPY = "JPY"

    # ML / LLM specific
    TOKENS = "TOKENS"
    TOKENS_PER_SECOND = "TOKENS_PER_SECOND"
    SAMPLES = "SAMPLES"
    SAMPLES_PER_SECOND = "SAMPLES_PER_SECOND"
    REQUESTS_PER_SECOND = "REQUESTS_PER_SECOND"
    FLOPS = "FLOPS"
    GFLOPS = "GFLOPS"
    TFLOPS = "TFLOPS"

In [15]:
from enum import Enum

class MathOperator(str, Enum):
    ADD = "ADD"
    SUBTRACT = "SUBTRACT"
    MULTIPLY = "MULTIPLY"
    DIVIDE = "DIVIDE"
    FLOOR_DIVIDE = "FLOOR_DIVIDE"
    MOD = "MOD"
    POWER = "POWER"
    ROOT = "ROOT"
    SQRT = "SQRT"
    FLOOR = "FLOOR"
    CEILING = "CEILING"
    MIN = "MIN"
    MAX = "MAX"
    

class Comparator(str, Enum):
    EQUAL_TO = "EQUAL_TO"
    NOT_EQUAL_TO = "NOT_EQUAL_TO"
    LESS_THAN = "LESS_THAN"
    LESS_THAN_OR_EQUAL_TO = "LESS_THAN_OR_EQUAL_TO"
    GREATER_THAN = "GREATER_THAN"
    GREATER_THAN_OR_EQUAL_TO = "GREATER_THAN_OR_EQUAL_TO"
    

In [16]:
from pydantic import BaseModel, Field, RootModel
from typing import Union, Literal
from enum import Enum

# Separate enums for clarity and type safety
# class MathOperator(str, Enum):
#     ADD = "ADD"
#     MULTIPLY = "MULTIPLY"
#     DIVIDE = "DIVIDE"
#     CEIL = "CEIL"
    
# class Comparator(str, Enum):
#     GREATER_THAN = "GREATER_THAN"
#     LESS_THAN = "LESS_THAN"
#     EQUAL_TO = "EQUAL_TO"

# class Units(str, Enum):
#     HOURS = "HOURS"
#     MINUTES = "MINUTES"
#     EUROS = "EUROS"
#     PERCENT = "PERCENT"
#     UNITLESS = "UNITLESS"
    
# --- Node Definitions ---

class ValueNode(BaseModel):
    type: Literal["VALUE"] = "VALUE"
    value: float
    description: str
    unit: Units

class CategoricalValueNode(BaseModel):
    type: Literal["CATEGORICAL_VALUE"] = "CATEGORICAL_VALUE"
    value: str
    description: str

class VariableNode(BaseModel):
    type: Literal["VARIABLE"] = "VARIABLE"
    name: str 
    description: str
    unit: Units

class BinaryOpNode(BaseModel):
    """Node for mathematical operations that produce a number."""
    type: Literal["BINARY_OPERATION"] = "BINARY_OPERATION"
    operator: MathOperator
    left: 'AnyNode'
    right: 'AnyNode'

class ComparisonNode(BaseModel):
    """Node for comparison operations that produce a boolean."""
    type: Literal["COMPARISON"] = "COMPARISON"
    operator: Comparator
    left: 'AnyNode'
    right: 'AnyNode'

class ConditionalNode(BaseModel):
    """Node for if-then-else logic."""
    type: Literal["CONDITIONAL"] = "CONDITIONAL"
    condition: ComparisonNode # Condition must be a comparison
    if_true: 'AnyNode'
    if_false: 'AnyNode'

# --- Recursive Setup ---

AnyNode = Union[
    ValueNode,
    CategoricalValueNode, 
    VariableNode, 
    BinaryOpNode, 
    ConditionalNode
]

# Use model_rebuild() to safely resolve all forward references
BinaryOpNode.model_rebuild()
ConditionalNode.model_rebuild()
ComparisonNode.model_rebuild()

class Node(RootModel):
    root: BinaryOpNode

In [17]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Dict, Any, Optional
from enum import Enum


class ParameterStatus(str, Enum):
    """An enumeration for clear, explicit parameter statuses."""
    KNOWN = "KNOWN"
    SYMBOLIC = "SYMBOLIC"

class VariableType(str, Enum):
    """An enumeration for clear, explicit parameter statuses."""
    CATEGORICAL = "CATEGORICAL"
    NUMERIC = "NUMERIC"

class ParameterDetail(BaseModel):
    """A structured model to describe each parameter identified from the query."""
    name: str = Field(..., description="The name of the parameter.")
    status: ParameterStatus = Field(..., description="Whether the parameter's value is known from the query or is a symbolic variable.")
    type_of_variable: VariableType = Field(..., description="whether the variable is numeric or categorical")
    # value: Optional[Any] = Field(None, description="The actual value of the parameter, if its status is 'KNOWN'. Must be null if status is 'SYMBOLIC'.")
    


class ReasoningSchemaStep1(BaseModel):
    query_parameters: List[ParameterDetail] = Field(
        ...,
        description="A structured list of all parameters identified from the query and their status."
    )
class ReasoningSchemaStep2(BaseModel):
    """
    A simplified schema for Step 2 that captures all constants and rules as a simple list of descriptive strings.
    """
    identified_constants_and_rules: List[str] = Field(
        ...,
        description="A comprehensive list of all facts, constants, and conditional rules extracted from the document that are necessary for the final calculation. Each string in the list should be a self-contained, clear statement. For example: 'The rate for a Narrow Satellite stand is €32.90 per 15 minutes' or 'A 100% surcharge is applied if parking duration is between 48 and 72 hours'."
    )
class ReasoningSchemaStep3(BaseModel):
    synthesis_plan: str = Field(
        ...,
        description="A concise, step-by-step plan describing how the variables and constants are combined into the final computation graph."
    )
class ReasoningSchemaStep4(BaseModel):
    rethink: str = Field(
        ...,
        description="Final check to ensure the plan correctly uses variables and constants and handles all logic from the document."
    )


In [18]:
@guidance
def create_graph_with_cot(llm, allowed_variables_prompt, thought1, document, query, output_schema):
    
    with system():
        llm += f"""You are an expert system that converts textual calculation rules into structured JSON expression trees.
        You MUST think step-by-step and reason before generating the final JSON.
        
        **Reasoning Guidelines:**
        1.  **Analyze Query Parameters:** Identify all relevant parameters from the user's query. For each parameter, create a structured object specifying its 'name', its 'status' ('KNOWN' if the value is given, or 'SYMBOLIC' if it's a variable), its 'type_of_variable' ('NUMERIC' if the value is a number or 'CATEGORICAL' if the value is categories), and its 'value' (or null if symbolic). For example: `[ {{"name": "aircraft_stand_type", "status": "KNOWN", "type_of_variable": "CATEGORICAL", "value": "Wide Remote"}}, {{"name": "parking_duration_hours", "status": "SYMBOLIC", "type_of_variable": "NUMERIC", "value": null}} ]`
        **Allowed Variables for this Task:**
        ---
        {allowed_variables_prompt}
        ---
        2. **Identify All Relevant Information**: Review the document and extract every fact, constant, and conditional rule needed for the calculation. Each piece of information should be written as a clear, self-contained sentence and collected into a list of strings.
        3.  **Synthesize Plan:** Briefly describe how you will combine these pieces into a final expression tree.
        4. **Rethink and Finalize Approach**: Before processing with generation, rethink your progress so far and make adjustments if necessary, then finalize and proceed to generate the expression tree.

        **Crucial Rule 1:** If a parameter from the 'Allowed Variables' list is given a specific value in the query, you MUST treat it as a fixed value to find constants. You MUST NOT include it as a `VARIABLE` node in the final JSON.
        **Crucial Rule 2**: If a calculation path or value depends on the value of a symbolic variable, you MUST capture the rules for all possible values and represent this logic using CONDITIONAL nodes in the final expression tree. You MUST NOT assume a default value for the variable to simplify the logic.
        **Crucial Rule 3 (Categorical Variables)**: Variables whose domain is categorical (e.g., string-valued like 'flight_type', 'stand_type') must be handled explicitly.
            - If the query supplies a category, treat it as fixed (per Rule 1).
            - If the query does not supply a category, mark the variable as 'SYMBOLIC' in Step 1 and ensure the final expression covers all category-dependent branches stated in the document using `CONDITIONAL` nodes. Do NOT assume a default category. Always use `CATEGORICAL_VALUE_NODE` for category constants.
            - Ensure every variable in the Allowed Variables list appears in Step 1 with a status of KNOWN or SYMBOLIC.
            - Encoding guidance: when checking a categorical branch, use a `COMPARISON` node with operator `EQUAL_TO`, `left` as a `VARIABLE` node for the categorical variable, and `right` as a `CATEGORICAL_VALUE_NODE` whose `value` is the category string (e.g., "International").

        **Crucial Rule 4 (Numeric vs Categorical Comparisons)**:
            - Numeric variables must be compared against numeric `VALUE` nodes whose `value` is a number.
            - Categorical variables must be compared against `CATEGORICAL_VALUE_NODE` nodes whose `value` is the category string.
            - Do NOT compare a categorical variable to a numeric `VALUE` node, and do NOT compare a numeric variable to a `CATEGORICAL_VALUE_NODE`.
            
        After writing your reasoning, you WILL generate the JSON object.


        """

    with user():
        llm += f"""
        **Document:**
        ---
        {document}
        ---

        **Query:**
        
        Based on the document, construct the computation graph for the following request:
        
        "{query}"

        ---

        **Initiate Process:**

        You will now follow the reasoning guidelines step-by-step before generating the final JSON.\n
        Step1. Analyze Query Parameters:\n

        {thought1}
        """

    # with assistant():
    #     # llm += "I will now follow the reasoning guidelines step-by-step before generating the final JSON.\n"
    #     # llm += "Step1. Analyze Query Parameters:\n"
    #     llm += guidance.json(
    #         name="thought1", 
    #         schema=ReasoningSchemaStep1, 
    #         max_tokens=1000)
    
    with user():
        llm += "Step2. Identify All Relevant Information:\n"

    with assistant():
        # llm += "Step2. Identify All Relevant Information:\n"
        llm += guidance.json(
            name="thought2", 
            schema=ReasoningSchemaStep2, 
            max_tokens=1000)

    with user():
        llm += "Step3. Synthesize Plan:\n"

    with assistant():
        # llm += "Step3. Synthesize Plan:\n"
        llm += guidance.json(
            name="thought3", 
            schema=ReasoningSchemaStep3, 
            max_tokens=1000)

    with user():
        llm += "Step4. Rethink and Finalize Approach:\n"

    with assistant():
        # llm += "Step4. Rethink and Finalize Approach:\n"
        llm += guidance.json(
            name="thought4", 
            schema=ReasoningSchemaStep4, 
            max_tokens=1000)

    with user():
        llm += "Step5. Generate the JSON object:\n"
        # After thinking, it generates the JSON.
        # llm += "\n\nFinal JSON object:\n"
        
    with assistant():
        llm += gen_json(
            name="result_graph", 
            schema=output_schema,
            max_tokens=8000 
        )
        
    return llm

In [19]:

class ComputationGraphBuilder:
    """
    Orchestrates the creation of a computation graph by preparing dynamic
    constraints and prompting the LLM.
    """
    
    def __init__(self, model):
        """
        Initializes the builder with a guidance model.
        """
        self.model = model
        # Set the default LLM for all guidance programs
        # guidance.llm = self.model
    @timing_decorator
    def build(self, document_content: str, query: str, charge_category: str, charge_category_variables: dict, all_variables: dict) -> dict:
        """
        Generates a computation graph for a given query and document.

        Args:
            document_content: The text containing the rules.
            query: A natural language question about what to calculate.
            charge_category: The specific charge context used to filter variables.

        Returns:
            A dictionary representing the computation graph or an error.
        """
        print(f"--- Building graph for charge category: '{charge_category}' ---")
        
        # 1. Dynamically create the filtered Enum for this specific task
        try:
            Var = create_dynamic_variable_enum(charge_category, charge_category_variables)
        except ValueError as e:
            print(f"Error: {e}")
            return {"error": str(e)}

        def _var_type(v):
            try:
                return "CATEGORICAL" if v.data_type is str else "NUMERIC"
            except Exception:
                return "UNKNOWN"
        # 3. Create a formatted prompt string of allowed variables for the LLM
        allowed_variables = [el.value for el in list(Var)]
        allowed_variables_prompt = "\n".join(
            [
                f"- **{v.name}** | type: {_var_type(v)} | unit: {v.unit} — {v.description}"
                for name, v in all_variables.items() if name in allowed_variables
            ]
        )

        allowed_variables_t1_struct = "\n".join(
            [
                f"""variable name={v.name}, \n type_of_variable={_var_type(v)}, \n status=SYMBOLIC, \n description={v.description}"""
                for name, v in all_variables.items() if name in allowed_variables
            ]
        )
        # {ParameterDetail(name=v.name, status="SYMBOLIC", type_of_variable=_var_type(v)) for name, v in all_variables.items() if name in allowed_variables}
        

        try:
            # 4. Execute the guidance program with all dynamic components
            result_lm = self.model + create_graph_with_cot(
                allowed_variables_prompt=allowed_variables_prompt,
                thought1 = allowed_variables_t1_struct,
                document=document_content,
                query=query,
                output_schema=Node
            )
            
            
            # print("\nSuccessfully generated graph:")
            # # Use model_dump_json for Pydantic v2
            # print(pydantic_graph.model_dump_json(indent=2)) 
            return result_lm
            
        except Exception as e:
            print(f"\nAn error occurred while building the graph for '{query}': {e}")
            return {"error": str(e)}

In [20]:
import sympy
import re, keyword
from sympy import (
    Symbol, Piecewise, sympify, Add, Mul, Pow,
    Min, Max, floor, ceiling, Rational, Mod
)

def _sanitize_symbol_name(val: object) -> str:
    # name = re.sub(r"\W+", "_", str(val)).strip("_")
    name = re.sub(r"[^A-Za-z0-9]+", "", str(val))
    if not name:
        name = "_VAL"
    # Must start with letter or underscore
    if not re.match(r"[A-Za-z_]\w*$", name):
        name = "_" + name
    # Avoid reserved/boolean-like names
    if keyword.iskeyword(name) or name in {"None", "True", "False", "true", "false", "none"}:
        name = "_" + name
    return name.lower()

def compose_expression(node: dict):
    """
    Recursively parses a JSON graph into a SymPy expression,
    preventing automatic simplification.

    Args:
        node: A dictionary representing a node in the computation graph.

    Returns:
        A non-evaluated sympy expression representing the computation.
    """

    node_type = node.get('type')

    if node_type == "VALUE":
        return sympify(node['value'])
    
    elif node_type == "CATEGORICAL_VALUE":
        
        sanitized = _sanitize_symbol_name(node['value'])
        return Symbol(sanitized)
        # return sympify(sanitized)

    elif node_type == "VARIABLE":
        return Symbol(node['name'])

    elif node_type == "BINARY_OPERATION":
        left = compose_expression(node['left'])
        right = compose_expression(node['right'])
        operator = node['operator']
        
        # Use class constructors with evaluate=False to prevent simplification
        if operator == "ADD":
            return Add(left, right, evaluate=False)
        elif operator == "MULTIPLY":
            return Mul(left, right, evaluate=False)
        elif operator == "SUBTRACT":
            return Add(left, Mul(-1, right, evaluate=False), evaluate=False)
        elif operator == "FLOOR_DIVIDE":
            q = Mul(left, Pow(right, -1, evaluate=False), evaluate=False)
            return floor(q, evaluate=False)
        elif operator == "MOD":
            return Mod(left, right, evaluate=False)
        elif operator == "POWER":
            return Pow(left, right, evaluate=False)
        elif operator == "ROOT":
            inv = Pow(right, -1, evaluate=False)
            return Pow(left, inv, evaluate=False)
        elif operator == "SQRT":
            return Pow(left, Rational(1, 2), evaluate=False)
        elif operator == "FLOOR":
            return floor(left, evaluate=False)
        elif operator == "CEILING":
            return ceiling(left, evaluate=False)
        elif operator == "MIN":
            return Min(left, right, evaluate=False)
        elif operator == "MAX":
            return Max(left, right, evaluate=False)
        elif operator == "DIVIDE":
            # Division (a/b) is represented as a * (b**-1)
            power = Pow(right, -1, evaluate=False)
            return Mul(left, power, evaluate=False)
        else:
            raise ValueError(f"Unsupported binary operator: {operator}")

    elif node_type == "COMPARISON":
        left = compose_expression(node['left'])
        right = compose_expression(node['right'])
        operator = node['operator']

        if operator == "GREATER_THAN":
            return left > right
        elif operator == "LESS_THAN":
            return left < right
        elif operator == "EQUAL_TO":
            return sympy.Eq(left, right, evaluate=False)
        elif operator == "NOT_EQUAL_TO":
            return sympy.Ne(left, right, evaluate=False)
        elif operator == "LESS_THAN_OR_EQUAL_TO":
            return sympy.Le(left, right, evaluate=False)
        elif operator == "GREATER_THAN_OR_EQUAL_TO":
            return sympy.Ge(left, right, evaluate=False)
        else:
            raise ValueError(f"Unsupported comparison operator: {operator}")

    elif node_type == "CONDITIONAL":
        condition = compose_expression(node['condition'])
        if_true_expr = compose_expression(node['if_true'])
        if_false_expr = compose_expression(node['if_false'])
        
        return Piecewise((if_true_expr, condition), (if_false_expr, True))

    else:
        raise ValueError(f"Unknown node type: {node_type}")

In [21]:
import json
import traceback


def create_computation_graph(model, query, charge_category, markdown_content, charge_category_variables, all_variables):
    graph_builder = ComputationGraphBuilder(model=model)

    start_time = time.perf_counter()
    llm_structured_response = graph_builder.build(
        document_content=markdown_content,
        query=query,
        charge_category=charge_category,
        charge_category_variables=charge_category_variables,
        all_variables=all_variables
    )
    end_time = time.perf_counter()
    build_time = end_time - start_time

    return llm_structured_response, build_time
    

# Tyler's Evals Trial 1

## Pre-Processing

In [22]:
from json import JSONDecodeError

def generate(path):
    computed_records = {}
    with open(path, "r", encoding="utf-8") as f:
        records = [json.loads(line) for line in f if line.strip()]
    
    for record_id, record in enumerate(records):
        if record_id >= 5:
            continue
        content, charges = record
        markdown_content = content
        charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
        computed_records[str(record_id)] = {}
        import builtins
        def to_built_in_type(type_str: str):
            t = getattr(builtins, type_str, None)
            if isinstance(t, type):
                return t
            raise ValueError(f"{type_str} is not a valid built-in type")

        charge_variables = {}
        all_variables = {}
        for charge, variables in charge_category_variables.items():
            charge_variables[charge] = list(variables.keys())
            for var, details in variables.items():
                all_variables[var] = DomainVariable(name=var,
                    description=details['description'],
                    unit=details['unit'],
                    data_type=to_built_in_type(details['dtype']))

        for charge_category in list(charge_variables.keys()):
            
            query = f"Calculate the total {charge_category}. Charge description - {charges[charge_category]['charge_description']}"    
            print(query, charge_category, charge_variables, all_variables)
            result, execution_time = create_computation_graph(base_lm, query, charge_category, markdown_content, charge_variables, all_variables)
            
            try:
                payload = {charge_category: {
                    'query': query,
                    'thought2':json.loads(result.get("thought2")),
                    'thought3':json.loads(result.get("thought3")),
                    'thought4':json.loads(result.get("thought4")),
                    'computation_graph':json.loads(result.get("result_graph")),
                    'execution_time': execution_time,
                    'status': 'SUCCESS'
                }
                }
                computed_records[str(record_id)][charge_category] = payload
            except JSONDecodeError:
                payload = {charge_category: {
                    'query': query,
                    'thought2':result.get("thought2"),
                    'thought3':result.get("thought3"),
                    'thought4':result.get("thought4"),
                    'computation_graph':result.get("result_graph"),
                    'execution_time': execution_time,
                    'status': 'ERROR'
                }
                }

            
    return computed_records

In [23]:
from sympy import lambdify, Symbol

def evaluate(computation_graph, charge_variable, ground_truth_code):
    symbolic_expr = compose_expression(computation_graph)
    
    charge, variables = charge_variable
    symbolic_vars = tuple([Symbol(var) for var in variables])
    
    symbolic_func = lambdify(symbolic_vars, symbolic_expr, 'sympy')

    ns = {}
    exec(ground_truth_code, ns)                 # defines compute_charge in ns
    ground_truth_func = ns['compute_charge']

    return symbolic_func, ground_truth_func


In [24]:
path = "/gpfs01/home/ppxac9/MLiS-Placement-Thesis/LLM_generated_data/synthetic_dataset/datapoints_8B.jsonl"
# path = "/gpfs01/home/ppxac9/MLiS-Placement-Thesis/LLM_generated_data/synthetic_dataset/datapoints_70B_4bit.jsonl"
result = generate(path)

Calculate the total compute_vm_fee. Charge description - Charge for running virtual machines based on vCPU, memory, OS, and commitment. compute_vm_fee {'compute_vm_fee': ['os_type', 'usage_hours'], 'gpu_compute_fee': ['gpu_type'], 'managed_disk_fee': ['managed_disk_count'], 'data_egress_fee': ['region'], 'data_ingress_fee': ['region', 'data_ingress_gb'], 'storage_transaction_fee': ['redundancy', 'storage_transactions_millions'], 'public_ip_fee': ['region', 'public_ip_count'], 'load_balancer_fee': ['load_balancer_rules'], 'aks_cluster_fee': ['region'], 'function_app_fee': ['region', 'function_exec_ms'], 'sql_database_fee': ['region', 'backup_retention_days'], 'snapshot_storage_fee': ['redundancy'], 'app_service_fee': ['service_tier', 'app_service_instances']} {'os_type': DomainVariable(name='os_type', description='Operating system for the compute workload', unit='os', data_type=<class 'str'>), 'usage_hours': DomainVariable(name='usage_hours', description='Total runtime in hours over the

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 25.4051 seconds
Calculate the total gpu_compute_fee. Charge description - Charge for GPU-enabled compute usage. gpu_compute_fee {'compute_vm_fee': ['os_type', 'usage_hours'], 'gpu_compute_fee': ['gpu_type'], 'managed_disk_fee': ['managed_disk_count'], 'data_egress_fee': ['region'], 'data_ingress_fee': ['region', 'data_ingress_gb'], 'storage_transaction_fee': ['redundancy', 'storage_transactions_millions'], 'public_ip_fee': ['region', 'public_ip_count'], 'load_balancer_fee': ['load_balancer_rules'], 'aks_cluster_fee': ['region'], 'function_app_fee': ['region', 'function_exec_ms'], 'sql_database_fee': ['region', 'backup_retention_days'], 'snapshot_storage_fee': ['redundancy'], 'app_service_fee': ['service_tier', 'app_service_instances']} {'os_type': DomainVariable(name='os_type', description='Operating system for the compute workload', unit='os', data_type=<class 'str'>), 'usage_hours': DomainVariable(name='usage_hours', description='Total runtime in hours ove

In [25]:
import pickle
with open('result_8b_4b_1.pkl', 'wb') as f:
    pickle.dump(result, f)


In [58]:
from json import JSONDecodeError

def generate_missing(path, _missing_charge, _doc_id):
    computed_records = {}
    with open(path, "r", encoding="utf-8") as f:
        records = [json.loads(line) for line in f if line.strip()]
    
    for record_id, record in enumerate(records):
        if record_id != _doc_id:
            continue
        content, charges = record
        markdown_content = content
        charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
        computed_records[str(record_id)] = {}
        import builtins
        def to_built_in_type(type_str: str):
            t = getattr(builtins, type_str, None)
            if isinstance(t, type):
                return t
            raise ValueError(f"{type_str} is not a valid built-in type")

        charge_variables = {}
        all_variables = {}
        for charge, variables in charge_category_variables.items():
            charge_variables[charge] = list(variables.keys())
            for var, details in variables.items():
                all_variables[var] = DomainVariable(name=var,
                    description=details['description'],
                    unit=details['unit'],
                    data_type=to_built_in_type(details['dtype']))

        for charge_category in list(charge_variables.keys()):
            if charge_category != _missing_charge:
                continue
            query = f"Calculate the total {charge_category}. Charge description - {charges[charge_category]['charge_description']}"    
            print(query, charge_category, charge_variables, all_variables)
            result, execution_time = create_computation_graph(base_lm, query, charge_category, markdown_content, charge_variables, all_variables)
            
            try:
                payload = {charge_category: {
                    'query': query,
                    'thought2':json.loads(result.get("thought2")),
                    'thought3':json.loads(result.get("thought3")),
                    'thought4':json.loads(result.get("thought4")),
                    'computation_graph':json.loads(result.get("result_graph")),
                    'execution_time': execution_time,
                    'status': 'SUCCESS'
                }
                }
                computed_records[str(record_id)][charge_category] = payload
            except JSONDecodeError:
                payload = {charge_category: {
                    'query': query,
                    'thought2':result.get("thought2"),
                    'thought3':result.get("thought3"),
                    'thought4':result.get("thought4"),
                    'computation_graph':result.get("result_graph"),
                    'execution_time': execution_time,
                    'status': 'ERROR'
                }
                }

            
    return computed_records

In [ ]:
path = "/gpfs01/home/ppxac9/MLiS-Placement-Thesis/LLM_generated_data/synthetic_dataset/datapoints_8B.jsonl"
missing_result = generate_missing(path, 'compute_vm_fee', 0)

Calculate the total load_balancer_fee. Charge description - Charge for load balancer rules and tier. load_balancer_fee {'compute_vm_fee': ['spot_instance'], 'gpu_compute_fee': ['os_type', 'gpu_count'], 'managed_disk_fee': ['managed_disk_count'], 'data_egress_fee': ['region'], 'data_ingress_fee': ['region'], 'storage_transaction_fee': ['redundancy', 'storage_transactions_millions'], 'public_ip_fee': ['region'], 'load_balancer_fee': ['region', 'load_balancer_rules'], 'aks_cluster_fee': ['region', 'kubernetes_node_count'], 'function_app_fee': ['region', 'function_exec_ms'], 'sql_database_fee': ['sql_storage_gb'], 'snapshot_storage_fee': ['redundancy'], 'app_service_fee': ['app_service_instances']} {'spot_instance': DomainVariable(name='spot_instance', description='Whether the compute uses spot capacity', unit='flag', data_type=<class 'str'>), 'os_type': DomainVariable(name='os_type', description='Operating system for the compute workload', unit='os', data_type=<class 'str'>), 'gpu_count':

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 22.0633 seconds


In [71]:
missing_result['4']['load_balancer_fee']

{'load_balancer_fee': {'query': 'Calculate the total load_balancer_fee. Charge description - Charge for load balancer rules and tier.',
  'thought2': {'identified_constants_and_rules': ['Region: Azure region where resources are deployed',
    'Load Balancer Rules: Configured load balancer rules',
    'Charge for load balancer rules and tier']},
  'thought3': {'synthesis_plan': 'The charge for load balancer rules and tier is calculated by multiplying the number of configured rules by a rate that depends on the region where the load balancer is deployed. The region is used to determine the rate, and the number of rules is used directly.'},
  'thought4': {'rethink': 'To calculate the total load_balancer_fee, we need to determine the charge for load balancer rules and tier. The charge depends on the region, and the number of rules is used directly. The region is a categorical variable, so we need to handle it with a conditional node to ensure all possible values are covered. The rules are 

In [79]:
result

{'0': {'compute_vm_fee': {'compute_vm_fee': {'query': 'Calculate the total compute_vm_fee. Charge description - Charge for running virtual machines based on vCPU, memory, OS, and commitment.',
    'thought2': {'identified_constants_and_rules': ['os_type: categorical, symbolic, description=Operating system for the compute workload',
      'usage_hours: numeric, symbolic, description=Total runtime in hours over the billing window',
      'compute_vm_fee: computed based on vCPU, memory, OS, and commitment']},
    'thought3': {'synthesis_plan': 'The compute_vm_fee is calculated based on the vCPU, memory, OS, and commitment parameters. The formula will take the values of os_type and usage_hours to determine the charge for virtual machines.'},
    'thought4': {'rethink': 'The parameters os_type and usage_hours are already identified as categorical and symbolic respectively, which allows their values to be treated as fixed constants. The compute_vm_fee is derived from the sum of vCPU, memory,

In [72]:
result['4']['load_balancer_fee'] =  missing_result['4']['load_balancer_fee']

In [73]:
import pickle 

with open('result_8b_0_6b_2.pkl', 'wb') as f:
    pickle.dump(result, f)

# with open('result_4.pkl', 'wb') as f:
#     pickle.dump(result, f)

# with open('result_4.pkl', 'rb') as f:
#     result = pickle.load(f)

# Testing

In [54]:

def get_symbolic_groundtruth_functions(path, result_path):
    master_function_list = {}
    error_list = {}
    with open(path, "r", encoding="utf-8") as f:
            records = [json.loads(line) for line in f if line.strip()]
    with open(result_path, 'rb') as f:
        result = pickle.load(f)
    for record_id, record in enumerate(records):
        content, charges = record
        markdown_content = content
        charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
        master_function_list[record_id] = {}
        error_list[record_id] = {}
        import builtins
        def to_built_in_type(type_str: str):
            t = getattr(builtins, type_str, None)
            if isinstance(t, type):
                return t
            raise ValueError(f"{type_str} is not a valid built-in type")

        charge_variables = {}
        all_variables = {}
        for charge, variables in charge_category_variables.items():
            charge_variables[charge] = list(variables.keys())
            for var, details in variables.items():
                all_variables[var] = DomainVariable(name=var,
                    description=details['description'],
                    unit=details['unit'],
                    data_type=to_built_in_type(details['dtype']))
            try:
                graph = result[str(record_id)][charge][charge]['computation_graph']
                code_str = charges[charge]['code']
            except KeyError as e:
                print(record_id, charge, result[str(record_id)].keys(), )
            try:
                symbolic_func, ground_truth_func = evaluate(graph, (charge, charge_variables[charge]), code_str)
                master_function_list[record_id][charge] = (symbolic_func, ground_truth_func, graph)
            except Exception  as e:
                error_list[record_id][charge] = [graph, e]
            
    return master_function_list, error_list

In [74]:
path = "/gpfs01/home/ppxac9/MLiS-Placement-Thesis/LLM_generated_data/synthetic_dataset/datapoints_8B.jsonl"
result_path='result_8b_0_6b_2.pkl'
functions, errors = get_symbolic_groundtruth_functions(path, result_path)

In [91]:
len(functions[4])

6

In [ ]:
# with open(path, "r", encoding="utf-8") as f:
#     records = [json.loads(line) for line in f if line.strip()]


In [ ]:
with open(path, "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f if line.strip()]

master_evals = {}
for record_id, record in enumerate(records):
    
    content, charges = record
    charge_category_variables = {k: v['variables_used'] for k, v in charges.items()}
    master_evals[record_id] = {}
    charge_variables = {}
    all_variables = {}
    for charge, variables in charge_category_variables.items():
        master_evals[record_id][charge] = []
        print(record_id, charge)
        if charge not in functions[record_id].keys():
            continue
        generated_func, ground_truth_func, graph = functions[record_id][charge]
        code_str = charges[charge]['code']
        tests = create_tests(code_str)
        ground_truth_func_evals = {}
        generated_func_test_evals = {}
        for condition_no, test in enumerate(tests):
            ground_truth_func_evals[condition_no] = [coerce_literals(test) for test in cartesian_combine_lists(*test)]
            generated_func_test_evals[condition_no] = symbolize_records([coerce_literals(test) for test in cartesian_combine_lists(*test)])

        ground_truth_computed_result = []
        symbolic_func_computed_result = []
        for i, condition in ground_truth_func_evals.items():
            for test in condition:
                result = ground_truth_func(**test)
                # if charge == 'function_app_fee' and record_id == 4:
                #     print(test, result)
                ground_truth_computed_result.append(result)

        for i, condition in generated_func_test_evals.items():
            for test in condition:
                result = generated_func(**test)
                # if charge == 'function_app_fee' and record_id == 4:
                #     print(">",test, result)
                symbolic_func_computed_result.append(result)
        exception_list = [(0, 'compute_vm_fee'), 
                        (0, 'data_ingress_fee'), 
                        (1, 'storage_transaction_fee'),
                        (2, 'public_ip_fee'),
                        (2, 'function_app_fee'),
                        (4, 'data_ingress_fee'),
                        (1, 'data_ingress_fee'),
                        (1, 'compute_vm_fee'),
                        (1, 'gpu_compute_fee'),
                        (1, 'public_ip_fee'),
                        (2, 'data_egress_fee'),
                        (3, 'aks_cluster_fee'),
                        (4, 'function_app_fee')]
        for symr, gtr in zip(symbolic_func_computed_result, ground_truth_computed_result):
            if isinstance(symr, float):
                master_evals[record_id][charge].append([symr, gtr, round(symr, 6) == round(gtr, 6)])
            else:
                master_evals[record_id][charge].append([symr, gtr, symr == gtr])
            if (record_id, charge) not in exception_list:    
                if isinstance(symr, float):
                    if round(symr, 6) != round(gtr, 6):
                        print(record_id, charge, symr, gtr)
                elif symr != gtr:
                    print(record_id, charge, symr, gtr)


0 compute_vm_fee
0 gpu_compute_fee
0 managed_disk_fee
0 data_egress_fee
0 data_ingress_fee
0 storage_transaction_fee
0 public_ip_fee
0 load_balancer_fee
0 aks_cluster_fee
0 function_app_fee
0 sql_database_fee
0 snapshot_storage_fee
0 app_service_fee
1 compute_vm_fee
1 gpu_compute_fee
1 managed_disk_fee
1 data_egress_fee
1 data_ingress_fee
1 storage_transaction_fee
1 public_ip_fee
1 load_balancer_fee
1 aks_cluster_fee
1 function_app_fee
1 function_app_fee 386230 386230
1 sql_database_fee
1 snapshot_storage_fee
1 app_service_fee
2 compute_vm_fee
2 gpu_compute_fee
2 managed_disk_fee
2 data_egress_fee
2 data_ingress_fee
2 storage_transaction_fee
2 public_ip_fee
2 load_balancer_fee
2 aks_cluster_fee
2 function_app_fee
2 sql_database_fee
2 snapshot_storage_fee
2 app_service_fee
3 compute_vm_fee
3 gpu_compute_fee
3 managed_disk_fee
3 data_egress_fee
3 data_ingress_fee
3 storage_transaction_fee
3 public_ip_fee
3 load_balancer_fee
3 aks_cluster_fee
3 function_app_fee
3 sql_database_fee
3 snapsh

In [78]:
functions[0].keys()

dict_keys(['managed_disk_fee', 'data_egress_fee', 'public_ip_fee', 'sql_database_fee', 'snapshot_storage_fee'])

In [136]:
functions[4]['function_app_fee'][0]

<function _lambdifygenerated(region, function_exec_ms)>

In [141]:
import pickle 

# with open('result_4.pkl', 'wb') as f:
#     pickle.dump(result, f)

with open('result_4.pkl', 'rb') as f:
    result = pickle.load(f)
compose_expression(result['4']['snapshot_storage_fee']['snapshot_storage_fee']['computation_graph'])

storage_volume*Piecewise((1, Eq(lrs, redundancy)), (Piecewise((2, Eq(gzrs, redundancy)), (Piecewise((3, Eq(grs, redundancy)), (Piecewise((4, Eq(redundancy, zrs)), (0, True)), True)), True)), True))

In [144]:
functions[4]['snapshot_storage_fee'][0]

<function _lambdifygenerated(redundancy)>

In [143]:
records[4][1]['snapshot_storage_fee']['code']

"def compute_charge(redundancy):\n    if redundancy == 'LRS': return 1\n    elif redundancy == 'GZRS': return 2\n    elif redundancy == 'GRS': return 3\n    elif redundancy == 'ZRS': return 4\n    else: raise ValueError(f'Unknown value for redundancy: {redundancy}')"

In [138]:
records[4][0]

'# Microsoft Azure Cloud Pricing Policy Document\n\nMicrosoft Azure Pricing Policy\n\nThis document provides an overview of the fee structure and pricing policies for Microsoft Azure cloud services, effective as of the current date. It outlines the terms and conditions that govern the pricing of Azure services in the United Kingdom (GBP) and is intended to provide clarity on the costs associated with utilizing Azure products and services. By utilizing Azure, customers agree to be bound by the terms and conditions outlined in this policy.\n\n## **Compute VM Pricing: Hourly and Monthly Charges for Virtual Machine Instances**\n\n<table border="1" cellspacing="0" cellpadding="6">\n  <thead>\n    <tr>\n      <th>Spot Flag</th>\n      <th>Charge</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>True</td>\n      <td>£1</td>\n    </tr>\n    <tr>\n      <td>False</td>\n      <td>£2</td>\n    </tr>\n  </tbody>\n</table>\n\n## **GPU Compute Fee Pricing**\n\nThis section outlines the pri

In [145]:
records[4][1]['snapshot_storage_fee']['variables_used']

{'redundancy': {'unit': 'redundancy',
  'description': 'Storage redundancy configuration',
  'dtype': 'str'}}

In [128]:
records[3][1]['data_egress_fee']['code']

"def compute_charge(region, data_egress_gb):\n    if region == 'uk_south' and 16366.2 <= data_egress_gb < 100000.0: return 1 * data_egress_gb * 1.0\n    elif region == 'uk_south' and 0.0 <= data_egress_gb < 16366.2: return 2 * data_egress_gb * 1.0\n    elif region == 'uk_west' and 16366.2 <= data_egress_gb < 100000.0: return 1 * data_egress_gb * 1.1\n    elif region == 'uk_west' and 0.0 <= data_egress_gb < 16366.2: return 2 * data_egress_gb * 1.1\n    elif region == 'westeurope' and 16366.2 <= data_egress_gb < 100000.0: return 1 * data_egress_gb * 1.2\n    elif region == 'westeurope' and 0.0 <= data_egress_gb < 16366.2: return 2 * data_egress_gb * 1.2\n    elif region == 'northeurope' and 16366.2 <= data_egress_gb < 100000.0: return 1 * data_egress_gb * 1.3\n    elif region == 'northeurope' and 0.0 <= data_egress_gb < 16366.2: return 2 * data_egress_gb * 1.3\n    elif region == 'eastus' and 16366.2 <= data_egress_gb < 100000.0: return 1 * data_egress_gb * 1.4\n    elif region == 'eastu

In [126]:
result['3']['data_egress_fee']['data_egress_fee']['thought4']

{'rethink': 'The plan is sound and fully compliant with the rules. The region is symbolic, so all six regions must be handled explicitly using conditional logic. The data_egress_gb is also symbolic, so the tiered pricing logic must be applied conditionally based on its value relative to 16,366.2 GB. The structure will use a top-level CATEGORICAL_VALUE_NODE for region comparison, with each branch leading to a nested NUMERIC_CONDITIONAL for the data volume threshold. The base and discounted rates for each region are fixed and derived directly from the document. The expression must be structured to compute the correct fee for any combination of region and data volume. No assumptions are made about defaults. All values are correctly encoded: region comparisons use CATEGORICAL_VALUE_NODE, numeric comparisons use VALUE nodes with numeric values. The final output will be a single JSON expression tree that evaluates to the total data_egress_fee in GBP. No simplifications or omissions are made.

In [93]:
import ast

def extract_if_conditions(code_str):
    """
    Extracts conditions from if-else statements in a Python function string.
    
    Args:
    code_str (str): The source code of the function as a string.
    
    Returns:
    list[str]: A list of condition strings (e.g., "x > 0" or "else").
    """
    try:
        tree = ast.parse(code_str)
    except SyntaxError as e:
        raise ValueError(f"Invalid Python code: {e}")
    
    class IfConditionVisitor(ast.NodeVisitor):
        def __init__(self):
            self.conditions = []
        
        def visit_If(self, node):
            current = node
            while True:
                self.conditions.append(ast.unparse(current.test).strip())
                
                # Visit statements in the body for nested structures
                for stmt in current.body:
                    self.visit(stmt)
                
                if current.orelse:
                    if len(current.orelse) == 1 and isinstance(current.orelse[0], ast.If):
                        # Handle elif by continuing the loop
                        current = current.orelse[0]
                        continue
                    else:
                        # Handle else
                        self.conditions.append("else")
                        # Visit statements in the else body
                        for stmt in current.orelse:
                            self.visit(stmt)
                
                # End of chain
                break
    
    visitor = IfConditionVisitor()
    visitor.visit(tree)
    
    return visitor.conditions

In [94]:
import re

def create_tests(ground_truth_code_string):
    binray_conditionals = ['and']
    numeric_conditionals = ['>=', "<=", "==", '<', '>']
    ops_pat = '|'.join(map(re.escape, numeric_conditionals))
    test_cases = []

    if_conditions = extract_if_conditions(ground_truth_code_string)
    
    for condition in if_conditions:
        condition_level_test = []
        expressions = [t.strip() for t in re.split(rf'\s*({'|'.join(binray_conditionals)})\b\s*', condition, flags=re.IGNORECASE)]
    
        for expression in expressions:
            
            if expression == 'else':
                continue    
            if expression == 'and':
                continue
            expression_components = [t.strip() for t in re.split(rf'\s*({ops_pat})\s*', expression)]
            # print(expression_components)
            test = parse_expression(expression_components)
            condition_level_test.append(test)
        if condition_level_test != []:
            test_cases.append(condition_level_test)
    return test_cases

In [95]:
import random
import math

def sample_values(tokens, n=5, seed=None, domain=None, integer=False):
    """
    tokens:
      - 5 tokens: [L, op1, var, op2, R]
      - 3 tokens: [var, op, value] OR [value, op, var]
    operators: <, <=, >, >=, ==
    Returns n samples satisfying the constraint. Floats by default; ints if integer=True.
    For single-sided constraints (3 tokens), pass domain=(lo, hi) to bound the other side.
    """
    if seed is not None:
        random.seed(seed)

    def is_number(s):
        try:
            float(s)
            return True
        except Exception:
            return False

    # Collect bounds as (value, inclusive) or None
    lower = None   # (lo_val, lo_inclusive)
    upper = None   # (hi_val, hi_inclusive)
    eq_value = None

    if len(tokens) == 5:
        L, op1, var, op2, R = tokens
        a = float(L)
        b = float(R)

        # Left comparison: L op1 var  -> constraint on var
        if op1 == '<':
            # L < var  => var > L  (lower, open)
            lower = (a, False)
        elif op1 == '<=':
            # L <= var => var >= L (lower, closed)
            lower = (a, True)
        elif op1 == '>':
            # L > var  => var < L  (upper, open)
            upper = (a, False)
        elif op1 == '>=':
            # L >= var => var <= L (upper, closed)
            upper = (a, True)
        elif op1 == '==':
            eq_value = a
        else:
            raise ValueError(f"Unsupported operator: {op1}")

        # Right comparison: var op2 R -> constraint on var
        if op2 == '<':
            # var < R  (upper, open)
            upper = (b, False) if upper is None else min(upper, (b, False), key=lambda t:(t[0], t[1]==False))
        elif op2 == '<=':
            # var <= R (upper, closed)
            upper = (b, True) if upper is None else min(upper, (b, True), key=lambda t:(t[0], t[1]==False))
        elif op2 == '>':
            # var > R  (lower, open)
            lower = (b, False) if lower is None else max(lower, (b, False), key=lambda t:(t[0], t[1]==True))
        elif op2 == '>=':
            # var >= R (lower, closed)
            lower = (b, True) if lower is None else max(lower, (b, True), key=lambda t:(t[0], t[1]==True))
        elif op2 == '==':
            if eq_value is not None and abs(eq_value - b) > 0:
                # contradictory equals
                return []
            eq_value = b
        else:
            raise ValueError(f"Unsupported operator: {op2}")

    elif len(tokens) == 3:
        x, op, y = tokens

        # Normalize to "var ? value" direction
        if is_number(x) and not is_number(y):
            # pattern: value op var  -> flip to var flip(op) value
            val = float(x)
            var = y
            flip = {'<': '>', '<=': '>=', '>': '<', '>=': '<=', '==': '=='}
            op = flip.get(op)
            if op is None:
                raise ValueError("Unsupported operator")
        elif not is_number(x) and is_number(y):
            # pattern: var op value (already normalized)
            var = x
            val = float(y)
        else:
            raise ValueError("Ambiguous tokens: need exactly one numeric and one variable")

        # Single-sided constraint -> set a bound
        if op == '<':
            upper = (val, False)
        elif op == '<=':
            upper = (val, True)
        elif op == '>':
            lower = (val, False)
        elif op == '>=':
            lower = (val, True)
        elif op == '==':
            eq_value = val
        else:
            raise ValueError(f"Unsupported operator: {op}")
    else:
        raise ValueError("Unexpected number of tokens")

    # Handle equality (must also satisfy other bounds if present)
    if eq_value is not None:
        v = float(eq_value)
        if lower is not None:
            lo, lo_inc = lower
            if not (v > lo or (lo_inc and v == lo)):
                return []
        if upper is not None:
            hi, hi_inc = upper
            if not (v < hi or (hi_inc and v == hi)):
                return []
        if integer:
            if abs(v - round(v)) > 1e-12:
                return []  # equality to a non-integer but integer requested
            v = int(round(v))
        return [v for _ in range(n)]

    # Fill missing side from domain (if needed)
    if lower is None or upper is None:
        if domain is None:
            raise ValueError("Single-sided constraint: please provide domain=(lo, hi)")
        dom_lo, dom_hi = map(float, domain)
        if lower is None:
            lower = (dom_lo, True)
        if upper is None:
            upper = (dom_hi, True)

    lo, lo_inc = lower
    hi, hi_inc = upper

    # Empty interval check
    if hi < lo or (hi == lo and (not lo_inc or not hi_inc)):
        return []

    if integer:
        # Convert to integer inclusive bounds
        lo_i = (math.floor(lo) + 1) if not lo_inc else math.ceil(lo)
        hi_i = (math.ceil(hi) - 1) if not hi_inc else math.floor(hi)
        if lo_i > hi_i:
            return []
        return [random.randint(lo_i, hi_i) for _ in range(n)]

    # Float sampling; nudge for open bounds
    eps = 1e-12 * max(1.0, abs(lo), abs(hi))
    lo_f = lo if lo_inc else lo + eps
    hi_f = hi if hi_inc else hi - eps
    if hi_f < lo_f:
        return []

    return [random.uniform(lo_f, hi_f) for _ in range(n)]

In [96]:
import itertools

def cartesian_combine_lists(*lists_of_dicts):
    """
    Each arg is a list[dict]. Returns list[dict] for the Cartesian product.
    If any list is empty -> returns [].
    """
    out = []
    for prod in itertools.product(*lists_of_dicts):
        d = {}
        for part in prod:
            d.update(part)  # later lists override earlier on key conflicts
        out.append(d)
    return out

In [97]:
import ast

def coerce_literals(rec):
    out = {}
    for k, v in rec.items():
        if isinstance(v, str):
            try:
                v = ast.literal_eval(v)  # "'linux'" -> "linux", "598.4" -> 598.4
            except Exception:
                pass
        out[k] = v
    return out

# results = [compute_charge(**coerce_literals(rec)) for rec in records]

In [98]:
from sympy import Symbol

def symbolize_records(records, only_keys=None, skip_numeric_like=True, in_place=False):
    """
    records: list of dicts
    only_keys: set/iterable of keys to convert; if None => convert all str values
    skip_numeric_like: don't convert strings that parse as numbers (e.g. "123.4")
    in_place: mutate input list if True; otherwise return a new list
    """
    def is_numlike(s):
        try:
            float(s)
            return True
        except Exception:
            return False

    target = records if in_place else [dict(r) for r in records]
    for rec in target:
        for k, v in list(rec.items()):
            if isinstance(v, str) and (only_keys is None or k in only_keys):
                if skip_numeric_like and is_numlike(v):
                    continue
                rec[k] = Symbol(_sanitize_symbol_name(v))
    return target

In [99]:
from typing import List

def parse_expression(expression: List):
    if '==' in expression:
        return [{expression[0]: expression[2]}]
    else:
        values = sample_values(expression, n=1, integer=True)

        if len(expression) == 5:
            cases = [{expression[2]: v} for v in values]
            return cases
        elif len(expression) == 3:
            cases = [{expression[0]: v} for v in values]
            return cases
        else:
            raise('Unexpected Expression Length')

In [64]:
print(ground_truth_func('windows', 100.0))
print(symbolic_func(Symbol('Windows'), 100.0))


440.00000000000006


NameError: name 'symbolic_func' is not defined

In [65]:
tests = create_tests(code_str)

final_test_list = {}
final_test_list_symbolic = {}
for condition_no, test in enumerate(tests):
    
    final_test_list[condition_no] = [coerce_literals(test) for test in cartesian_combine_lists(*test)]
    final_test_list_symbolic[condition_no] = symbolize_records([coerce_literals(test) for test in cartesian_combine_lists(*test)])

In [66]:
ground_truth_result = []
symbolic_func_result = []
for i, condition in final_test_list.items():
    for test in condition:
        ground_truth_result.append(ground_truth_func(**test))
        

for i, condition in final_test_list_symbolic.items():
    for test in condition:
        symbolic_func_result.append(symbolic_func(**test))
        


NameError: name 'symbolic_func' is not defined

In [67]:
for symr, gtr in zip(symbolic_func_result, ground_truth_result):
    print(symr, gtr, symr == gtr)

In [32]:
final_test_list_symbolic

{0: [{'os_type': Linux, 'usage_hours': 665.3248833178142}],
 1: [{'os_type': Linux, 'usage_hours': 432.13550835119906}],
 2: [{'os_type': Linux, 'usage_hours': 350.7458075164126}],
 3: [{'os_type': Linux, 'usage_hours': 171.49853195365063}],
 4: [{'os_type': Windows, 'usage_hours': 690.057889740819}],
 5: [{'os_type': Windows, 'usage_hours': 554.5396517677665}],
 6: [{'os_type': Windows, 'usage_hours': 403.4211129037155}],
 7: [{'os_type': Windows, 'usage_hours': 113.57826201135656}]}

In [43]:
compose_expression(result['0']['compute_vm_fee']['compute_vm_fee']['computation_graph'])

usage_hours*Piecewise((Piecewise((4.0, usage_hours < 216.5), (Piecewise((3.0, usage_hours < 408.7), (Piecewise((2.0, usage_hours < 598.4), (1.0, True)), True)), True)), Eq(Linux, os_type)), (Piecewise((Piecewise((4.8, usage_hours < 216.5), (Piecewise((3.9, usage_hours < 408.7), (Piecewise((2.2, usage_hours < 598.4), (1.1, True)), True)), True)), Eq(Windows, os_type)), (0.0, True)), True))

In [53]:
records[0][1]['compute_vm_fee']['code']

"def compute_charge(os_type, usage_hours):\n    if os_type == 'linux' and 598.4 <= usage_hours < 744.0: return 1 * usage_hours * 1.0\n    elif os_type == 'linux' and 408.7 <= usage_hours < 598.4: return 2 * usage_hours * 1.0\n    elif os_type == 'linux' and 216.5 <= usage_hours < 408.7: return 3 * usage_hours * 1.0\n    elif os_type == 'linux' and 0.0 <= usage_hours < 216.5: return 4 * usage_hours * 1.0\n    elif os_type == 'windows' and 598.4 <= usage_hours < 744.0: return 1 * usage_hours * 1.1\n    elif os_type == 'windows' and 408.7 <= usage_hours < 598.4: return 2 * usage_hours * 1.1\n    elif os_type == 'windows' and 216.5 <= usage_hours < 408.7: return 3 * usage_hours * 1.1\n    elif os_type == 'windows' and 0.0 <= usage_hours < 216.5: return 4 * usage_hours * 1.1\n    else: raise ValueError(f'os_type or usage_hours is out of expected range')"

In [ ]:
# import builtins
# def to_built_in_type(type_str: str):
#     t = getattr(builtins, type_str, None)
#     if isinstance(t, type):
#         return t
#     raise ValueError(f"{type_str} is not a valid built-in type")

In [ ]:
# charge_variables = {}
# all_variables = {}
# for charge, variables in charge_category_variables.items():
#     charge_variables[charge] = list(variables.keys())
#     for var, details in variables.items():
#         all_variables[var] = DomainVariable(name=var,
#             description=details['description'],
#             unit=details['unit'],
#             data_type=to_built_in_type(details['dtype']))
    
    

In [ ]:
# charge_variables

{'compute_vm_fee': ['os_type', 'usage_hours'],
 'gpu_compute_fee': ['gpu_type'],
 'managed_disk_fee': ['managed_disk_count'],
 'data_egress_fee': ['region'],
 'data_ingress_fee': ['region', 'data_ingress_gb'],
 'storage_transaction_fee': ['redundancy', 'storage_transactions_millions'],
 'public_ip_fee': ['region', 'public_ip_count'],
 'load_balancer_fee': ['load_balancer_rules'],
 'aks_cluster_fee': ['region'],
 'function_app_fee': ['region', 'function_exec_ms'],
 'sql_database_fee': ['region', 'backup_retention_days'],
 'snapshot_storage_fee': ['redundancy'],
 'app_service_fee': ['service_tier', 'app_service_instances']}

In [ ]:
# all_variables

{'os_type': DomainVariable(name='os_type', description='Operating system for the compute workload', unit='os', data_type=<class 'str'>),
 'usage_hours': DomainVariable(name='usage_hours', description='Total runtime in hours over the billing window', unit='hours', data_type=<class 'float'>),
 'gpu_type': DomainVariable(name='gpu_type', description='Type of GPU accelerator', unit='gpu type', data_type=<class 'str'>),
 'managed_disk_count': DomainVariable(name='managed_disk_count', description='Count of managed disks attached', unit='disks', data_type=<class 'int'>),
 'region': DomainVariable(name='region', description='Azure region where resources are deployed', unit='region', data_type=<class 'str'>),
 'data_ingress_gb': DomainVariable(name='data_ingress_gb', description='Inbound data transferred to Azure', unit='GB', data_type=<class 'float'>),
 'redundancy': DomainVariable(name='redundancy', description='Storage redundancy configuration', unit='redundancy', data_type=<class 'str'>),
 

## Computation Graph Creation [OLD]

In [ ]:
# list(charge_variables.keys())[0]

'compute_vm_fee'

In [ ]:
# charges[list(charge_variables.keys())[0]]['charge_description']

'Charge for running virtual machines based on vCPU, memory, OS, and commitment.'

In [ ]:
# query = f"Calculate the total {list(charge_variables.keys())[0]}. Charge description - {charges[list(charge_variables.keys())[0]]['charge_description']}"
# charge_category = list(charge_variables.keys())[0]
# markdown_content = content
# result = create_computation_graph(base_lm, query, charge_category, markdown_content, charge_variables, all_variables)


--- Building graph for charge category: 'compute_vm_fee' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 116.5014 seconds


## Lookthrough [OLD]

In [ ]:
# print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

{
    "query_parameters": [
        {
            "name": "os_type",
            "status": "SYMBOLIC",
            "type_of_variable": "CATEGORICAL",
            "value": null
        },
        {
            "name": "usage_hours",
            "status": "SYMBOLIC",
            "type_of_variable": "NUMERIC",
            "value": null
        }
    ]
}


In [ ]:
# print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

{
    "identified_constants_and_rules": [
        "The compute VM pricing is based on the operating system (OS) and the number of usage hours.",
        "For Linux VMs, the hourly rate depends on the usage hours as follows: $4.00 per hour for 0 \u2013 216.5 hours, $3.00 per hour for 216.5 \u2013 408.7 hours, $2.00 per hour for 408.7 \u2013 598.4 hours, and $1.00 per hour for 598.4 \u2013 744.0 hours.",
        "For Windows VMs, the hourly rate depends on the usage hours as follows: $4.80 per hour for 0 \u2013 216.5 hours, $3.90 per hour for 216.5 \u2013 408.7 hours, $2.20 per hour for 408.7 \u2013 598.4 hours, and $1.10 per hour for 598.4 \u2013 744.0 hours.",
        "The compute_vm_fee is calculated as the product of the usage_hours and the applicable hourly rate based on the OS and the usage_hours range.",
        "The hourly rates are tiered and non-overlapping, with each tier applying to a specific range of usage_hours.",
        "The total compute_vm_fee is determined by selectin

In [ ]:
# print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

{
    "synthesis_plan": "To compute the total compute_vm_fee, we need to determine the applicable hourly rate based on the OS type (Linux or Windows) and the usage_hours. Since both os_type and usage_hours are symbolic (not provided with specific values), we must create a conditional expression tree that handles all possible combinations. The logic will first check the OS type using a CONDITIONAL node. For each OS (Linux and Windows), we will evaluate the usage_hours against the tiered ranges (0\u2013216.5, 216.5\u2013408.7, 408.7\u2013598.4, 598.4\u2013744.0) using nested CONDITIONAL nodes. Each range will return the corresponding hourly rate. The final result is the product of the selected hourly rate and the usage_hours. The expression tree will use COMPARISON nodes with appropriate operators (LESS_THAN_EQUAL, GREATER_THAN) and VALUE nodes for numeric thresholds. CATEGORICAL_VALUE_NODE will be used for OS comparisons."
}


In [ ]:
# print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

{
    "rethink": "The plan is sound and aligns with all crucial rules. Both 'os_type' and 'usage_hours' are symbolic, so we must not assume default values. The logic must cover all OS types (Linux/Windows) and all usage hour tiers. The tiered pricing is non-overlapping and cumulative, so we use a cascading conditional structure: first check OS, then check usage_hours against the increasing thresholds. The comparison for numeric ranges uses proper VALUE nodes (with numeric values), and categorical comparisons use CATEGORICAL_VALUE_NODE. The final multiplication is applied only after selecting the correct rate. No assumptions about defaults are made. The expression tree will correctly represent the full conditional logic required for any input. Finalization complete."
}


In [ ]:
# print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "MULTIPLY",
    "left": {
        "type": "VARIABLE",
        "name": "usage_hours",
        "description": "Total runtime in hours over the billing window",
        "unit": "HOURS"
    },
    "right": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "EQUAL_TO",
            "left": {
                "type": "VARIABLE",
                "name": "os_type",
                "description": "Operating system for the compute workload",
                "unit": "COUNT"
            },
            "right": {
                "type": "CATEGORICAL_VALUE",
                "value": "Linux",
                "description": "Operating system type: Linux"
            }
        },
        "if_true": {
            "type": "CONDITIONAL",
            "condition": {
                "type": "COMPARISON",
                "operator": "LESS_THAN",
                "left": {
                    "type": "

dict_keys(['compute_vm_fee', 'gpu_compute_fee', 'managed_disk_fee', 'data_egress_fee', 'data_ingress_fee', 'storage_transaction_fee', 'public_ip_fee', 'load_balancer_fee', 'aks_cluster_fee', 'function_app_fee', 'sql_database_fee', 'snapshot_storage_fee', 'app_service_fee'])

In [154]:
symbolic_expr = compose_expression(json.loads(result[0]["result_graph"]))

# Define the symbol that you want to replace
# variable_to_replace = Symbol('os_type')

# # Define the value you want to substitute
# value_to_substitute = 60

# final_charge_cost = symbolic_expr.subs(variable_to_replace, value_to_substitute)
# print(f"The parking cost for {value_to_substitute} hours is: {final_charge_cost} EUR") 

In [156]:
symbolic_expr

usage_hours*Piecewise((Piecewise((4.0, usage_hours < 216.5), (Piecewise((3.0, usage_hours < 408.7), (Piecewise((2.0, usage_hours < 598.4), (1.0, True)), True)), True)), Eq(Linux, os_type)), (Piecewise((Piecewise((4.8, usage_hours < 216.5), (Piecewise((3.9, usage_hours < 408.7), (Piecewise((2.2, usage_hours < 598.4), (1.1, True)), True)), True)), Eq(Windows, os_type)), (0.0, True)), True))

In [162]:
from sympy import lambdify, Symbol
os_type     = Symbol('os_type')
usage_hours = Symbol('usage_hours')


f = lambdify((os_type, usage_hours), symbolic_expr, 'sympy')

print(f(Symbol('Windows'), 220.0))


858.0


In [163]:
code_str

"def compute_charge(os_type, usage_hours):\n    if os_type == 'linux' and 598.4 <= usage_hours < 744.0: return 1 * usage_hours * 1.0\n    elif os_type == 'linux' and 408.7 <= usage_hours < 598.4: return 2 * usage_hours * 1.0\n    elif os_type == 'linux' and 216.5 <= usage_hours < 408.7: return 3 * usage_hours * 1.0\n    elif os_type == 'linux' and 0.0 <= usage_hours < 216.5: return 4 * usage_hours * 1.0\n    elif os_type == 'windows' and 598.4 <= usage_hours < 744.0: return 1 * usage_hours * 1.1\n    elif os_type == 'windows' and 408.7 <= usage_hours < 598.4: return 2 * usage_hours * 1.1\n    elif os_type == 'windows' and 216.5 <= usage_hours < 408.7: return 3 * usage_hours * 1.1\n    elif os_type == 'windows' and 0.0 <= usage_hours < 216.5: return 4 * usage_hours * 1.1\n    else: raise ValueError(f'os_type or usage_hours is out of expected range')"

In [144]:
code_str = charges[list(charge_variables.keys())[0]]['code']

ns = {}
exec(code_str, ns)                 # defines compute_charge in ns
compute_charge = ns['compute_charge']

# use it
print(compute_charge('linux', 300.0))      # example
print(compute_charge('windows', 220.0)) 

900.0
726.0000000000001


In [ ]:
charge_variables

{'compute_vm_fee': ['os_type', 'usage_hours'],
 'gpu_compute_fee': ['gpu_type'],
 'managed_disk_fee': ['managed_disk_count'],
 'data_egress_fee': ['region'],
 'data_ingress_fee': ['region', 'data_ingress_gb'],
 'storage_transaction_fee': ['redundancy', 'storage_transactions_millions'],
 'public_ip_fee': ['region', 'public_ip_count'],
 'load_balancer_fee': ['load_balancer_rules'],
 'aks_cluster_fee': ['region'],
 'function_app_fee': ['region', 'function_exec_ms'],
 'sql_database_fee': ['region', 'backup_retention_days'],
 'snapshot_storage_fee': ['redundancy'],
 'app_service_fee': ['service_tier', 'app_service_instances']}

In [ ]:
charge_variables['compute_vm_fee']

['os_type', 'usage_hours']

# Archive

In [20]:
query = "Calculate the total parking charge for a Narrow Satellite stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category, markdown_content, CHARGE_CATEGORY_VARIABLES, ALL_VARIABLES)



--- Building graph for charge category: 'east_aerodrome_parking_charge' ---


/gpfs01/home/ppxac9/MLiS-Placement-Thesis/pipeline/.venv/lib/python3.11/site-packages/guidance/models/_transformers.py:522: UserWarning: Cache is too small. Resetting cache (no method implemented to resize cache for type <class 'transformers.cache_utils.DynamicCache'>).
  warnings.warn(


KeyboardInterrupt: 

In [71]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

{
    "query_parameters": [
        {
            "name": "aircraft_stand_type",
            "status": "KNOWN",
            "value": "Narrow Satellite"
        },
        {
            "name": "parking_duration_hours",
            "status": "SYMBOLIC",
            "value": null
        },
        {
            "name": "is_overnight_parking",
            "status": "KNOWN",
            "value": false
        }
    ]
}


In [72]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

{
    "identified_constants_and_rules": [
        "The standard charge per aircraft/stand type for Narrow Satellite at East Aerodrome Parking (EAP) is \u20ac32.90 per 15 minutes or part thereof.",
        "Parking surcharges apply for extended periods at EAP: 48 hours up to 72 hours (including night-time) attract a surcharge of Standard rate +100%, and 72 hours and over (including night-time) attract a surcharge of Standard rate +200%.",
        "Overnight parking from 2300-0600hrs is free of charge. The query specifies no overnight parking (is_overnight_parking = false).",
        "The parking duration is a variable with unit hours, meaning the calculation must account for all possible values of parking_duration_hours."
    ]
}


In [73]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

{
    "synthesis_plan": "The total parking charge calculation for a Narrow Satellite stand at EAP involves: 1. Determining the base rate (\u20ac32.90 per 15 minutes or part thereof). 2. Calculating the number of 15-minute intervals in the parking duration. 3. Applying surcharges based on the duration: - If parking_duration_hours < 48: no surcharge. - If 48 \u2264 parking_duration_hours < 72: add 100% surcharge. - If parking_duration_hours \u2265 72: add 200% surcharge. 4. Since the duration is symbolic, the expression tree must include conditional logic for all possible duration ranges."
}


In [74]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

{
    "rethink": "The approach correctly identifies the base rate, the surcharge conditions based on duration, and the need for conditional logic due to the symbolic duration. The is_overnight_parking parameter is set to false, so no adjustment for free overnight parking is needed. The expression tree will accurately represent the calculation logic for any given parking duration."
}


In [75]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "LESS_THAN",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 48,
                "description": "Threshold for 48-hour surcharge.",
                "unit": "HOURS"
            }
        },
        "if_true": {
            "type": "BINARY_OPERATION",
            "operator": "MULTIPLY",
            "left": {
                "type": "VALUE",
                "value": 32.9,
                "description": "Standard charge per aircraft/stand type for Narrow Satellite at East Aerodrome Parking (EAP).",
                "unit"

In [76]:
result[1]

352.01867974799825

In [79]:



# Print the resulting expression
loaded_json = json.loads(result[0]['result_graph'])
symbolic_expr = compose_expression(loaded_json)
print(f"Sympy Expression: {symbolic_expr}")

Sympy Expression: Piecewise((32.9*(parking_duration_hours/0.25), parking_duration_hours < 48), (Piecewise(((32.9*2)*(parking_duration_hours/0.25), parking_duration_hours < 72), ((32.9*3)*(parking_duration_hours/0.25), True)), True)) + 0



## Changes: 
- Tried on a new problem

## Observations: 
- The reasoning traces are decent, except that I may need to add a ceil function to account for cases when one may need to round up.
- The graph produced is not perfect. It makes a potential mistake in computing surcharge, however, at this point, it is not clear whether this is a mistake as the document is quite confusing by itself. It does not clarify whether the surcharge is applicable for Long Term Remote, and whether that is on a per day basis.
- The left hand side branch does not use the variable parking hours to compute the surcharge even in cases where the hours are > 24. One potential reason for this could be that I mentioned there is no overnight parking, which may have led it to assume that the hours would be less than 24.
- The overnight parking charges also have ambiguity in their application. It is not clear, whether one has to discount the free period from the computed total fees or whether the surcharges are still applicable during that period, or it could be that aircrafts which strictly use the airport services starting and ending strictly during this period are exempt from any fees? 

## Discussion:
- More testing is required to understand where the model may be making mistakes.
- Larger models may perform better, as we currently only use the humble 32B A3B parameter MoE model.

## Next Steps:
- Continue testing

In [16]:
query = "Calculate the total parking charge for a Long Term Remote stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


--- Building graph for charge category: 'east_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 213.9923 seconds


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

{
    "query_parameters": [
        {
            "name": "aircraft_stand_type",
            "status": "KNOWN",
            "value": "Long Term Remote"
        },
        {
            "name": "parking_duration_hours",
            "status": "SYMBOLIC",
            "value": null
        },
        {
            "name": "is_overnight_parking",
            "status": "KNOWN",
            "value": false
        }
    ]
}


In [18]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

{
    "identified_constants_and_rules": [
        "The standard charge per aircraft/stand type for Long Term Remote at East Aerodrome Parking (EAP) is \u20ac234.50 per day or part thereof.",
        "Aircraft parking for extended periods in EAP attract surcharges based on duration: 48-72 hours (including night-time) is standard rate +100%, 72 hours and over is standard rate +200%.",
        "Overnight parking from 2300-0600hrs is free of charge.",
        "The parking duration is a variable with unit hours.",
        "The parking duration is not during overnight hours (is_overnight_parking = false).",
        "The parking is at East Aerodrome Parking (EAP).",
        "The stand type is Long Term Remote."
    ]
}


In [19]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

{
    "synthesis_plan": "The total parking charge calculation will start with the standard rate for Long Term Remote at EAP (\u20ac234.50/day). Since the duration is variable, we need to create conditional branches based on the duration in hours. The surcharge percentages depend on whether the duration is between 48-72 hours or over 72 hours. We'll convert the duration from hours to days (considering part thereof) and apply the appropriate surcharge. Since it's not overnight parking, we don't need to adjust for free night-time periods."
}


In [20]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

{
    "rethink": "The approach correctly identifies the standard rate and the surcharge conditions based on duration. The conversion from hours to days needs to be handled as 'per day or part thereof' which means any fraction of a day counts as a full day. The surcharge conditions are correctly identified as dependent on the duration. The is_overnight_parking parameter is correctly set to false so no adjustments for free overnight periods are needed."
}


In [21]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "GREATER_THAN",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 72,
                "description": "Threshold for 200% surcharge",
                "unit": "HOURS"
            }
        },
        "if_true": {
            "type": "BINARY_OPERATION",
            "operator": "MULTIPLY",
            "left": {
                "type": "VALUE",
                "value": 234.5,
                "description": "Standard charge per day for Long Term Remote at East Aerodrome Parking",
                "unit": "EUROS"
            }


## Changes: 
- Tried on a new problem

## Observations: 

## Discussion:

## Next Steps:


In [22]:
query = "Calculate the total parking charge for a Wide Satellite stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


--- Building graph for charge category: 'east_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 298.1813 seconds


In [23]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

{
    "query_parameters": [
        {
            "name": "aircraft_stand_type",
            "status": "KNOWN",
            "value": "Wide Satellite"
        },
        {
            "name": "parking_duration_hours",
            "status": "SYMBOLIC",
            "value": null
        },
        {
            "name": "is_overnight_parking",
            "status": "KNOWN",
            "value": false
        }
    ]
}


In [24]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

{
    "identified_constants_and_rules": [
        "The standard charge per aircraft/stand type for Wide Satellite at East Aerodrome Parking (EAP) is \u20ac42.00 per 15 minutes or part thereof.",
        "Parking surcharges apply for extended periods at EAP: 48 hours up to 72 hours (including night-time) attract a surcharge of Standard rate +100%, and 72 hours and over (including night-time) attract Standard rate +200%.",
        "Overnight parking from 2300-0600hrs is free of charge, but the query specifies no overnight parking (is_overnight_parking = false).",
        "The parking duration is a variable with unit hours, meaning the calculation must account for all possible values of parking_duration_hours.",
        "The parking charge is calculated based on the number of 15-minute intervals in the parking duration, with each interval charged at the standard rate or surcharged rate depending on duration.",
        "The minimum charge for light aircraft is \u20ac3.20, but this does not

In [25]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

{
    "synthesis_plan": "The total parking charge for a Wide Satellite stand at EAP depends on the parking duration. The base rate is \u20ac42.00 per 15 minutes or part thereof. For durations between 48 and 72 hours, the rate is doubled (100% surcharge), and for durations over 72 hours, the rate is tripled (200% surcharge). Since the parking duration is symbolic, the calculation must include conditional logic to handle all possible values of parking_duration_hours. The formula will calculate the number of 15-minute intervals, apply the appropriate rate based on duration, and sum the charges."
}


In [26]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

{
    "rethink": "The approach correctly identifies the base rate, surcharge conditions, and the need for conditional logic due to the symbolic parking duration. The calculation must account for the number of 15-minute intervals, apply surcharges based on duration, and handle all possible values of parking_duration_hours. The final expression tree will include conditional nodes for the surcharge tiers and a calculation for the total charge."
}


In [27]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "LESS_THAN",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 48,
                "description": "Threshold for 100% surcharge on parking duration.",
                "unit": "HOURS"
            }
        },
        "if_true": {
            "type": "BINARY_OPERATION",
            "operator": "MULTIPLY",
            "left": {
                "type": "BINARY_OPERATION",
                "operator": "DIVIDE",
                "left": {
                    "type": "VARIABLE",
                    "name": "parking_duration_hou

In [ ]:
query = "Calculate the total parking charge for a Light Aircraft Parking (LAP) stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


--- Building graph for charge category: 'east_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 839.8929 seconds


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

{
    "query_parameters": [
        {
            "name": "aircraft_stand_type",
            "status": "KNOWN",
            "value": "Light Aircraft Parking (LAP)"
        },
        {
            "name": "parking_duration_hours",
            "status": "SYMBOLIC",
            "value": null
        },
        {
            "name": "is_overnight_parking",
            "status": "KNOWN",
            "value": false
        }
    ]
}


In [21]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

{
    "identified_constants_and_rules": [
        "The parking charge for Light Aircraft Parking (LAP) at East Aerodrome Parking (EAP) is \u20ac3.50 per 15 minutes or part thereof.",
        "The parking duration is a variable with unit hours.",
        "Overnight parking from 2300-0600hrs is free of charge, but the query specifies no overnight parking.",
        "Aircraft parking for extended periods in EAP attract surcharges: 48 hours up to 72 hours (including night-time) has a surcharge of Standard rate +100%, and 72 hours and over has Standard rate +200%.",
        "The parking charge is calculated from the actual time of arrival to the actual time of departure minus 30 minutes.",
        "The minimum charge of \u20ac3.20 applies for light aircraft.",
        "The parking duration is a variable, so we need to account for all possible values."
    ]
}


In [22]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

{
    "synthesis_plan": "The total parking charge for LAP at EAP is calculated by first determining the base rate of \u20ac3.50 per 15 minutes or part thereof. Since the parking duration is a variable, we need to calculate the number of 15-minute intervals. However, we must also consider the surcharges for durations over 48 and 72 hours. The calculation should include a conditional check for the duration: if it's less than 48 hours, use the base rate; if between 48-72 hours, apply 100% surcharge; if over 72 hours, apply 200% surcharge. Additionally, the minimum charge of \u20ac3.20 must be considered, but since the base rate is already higher than this, it may not affect the calculation. The overnight parking is not applicable here as per the query."
}


In [23]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

{
    "rethink": "The approach is correct, but I need to ensure that the surcharge conditions are properly represented as conditional nodes in the expression tree. Also, the calculation of 15-minute intervals needs to be accurately represented. The minimum charge of \u20ac3.20 is lower than the base rate, so it should be considered as a minimum floor in the calculation."
}


In [25]:
result[0]["result_graph"]

'{"type": "BINARY_OPERATION", "operator": "ADD", "left": {"type": "CONDITIONAL", "condition": {"type": "COMPARISON", "operator": "GREATER_THAN", "left": {"type": "VARIABLE", "name": "parking_duration_hours", "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.", "unit": "HOURS"}, "right": {"type": "VALUE", "value": 72, "description": "Threshold for 200% surcharge on parking duration.", "unit": "HOURS"}}, "if_true": {"type": "BINARY_OPERATION", "operator": "MULTIPLY", "left": {"type": "VALUE", "value": 3.50, "description": "Standard charge per 15 minutes or part thereof for Light Aircraft Parking (LAP) at East Aerodrome Parking (EAP).", "unit": "EUROS"}, "right": {"type": "BINARY_OPERATION", "operator": "DIVIDE", "left": {"type": "VARIABLE", "name": "parking_duration_hours", "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.", "unit": "HOURS"}, "right": {"type": "VALUE", "value": 0.25, "description": "Conversion f

In [50]:
query = "Calculate the total parking charge for a Light Aircraft Parking (LAP) stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


--- Building graph for charge category: 'east_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

Execution time for 'build': 404.2457 seconds


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [53]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

{
    "rethink": "The query specifies that the parking duration is a variable with unit hours, and we must account for all possible values. The surcharge rules are based on duration thresholds (48 and 72 hours), so we must use conditional logic to handle these cases. The base rate is \u20ac3.50 per 15 minutes, and since the charge is per 15 minutes or part thereof, we must round up the duration to the next 15-minute interval. The minimum charge of \u20ac3.20 is already covered by the base rate, so it does not need to be explicitly applied. The 'no overnight parking' condition means we do not apply the free overnight period, so the full duration is charged. The final expression tree must include:\n- A function to convert hours to 15-minute intervals (ceiling division by 0.25)\n- Conditional logic for surcharges based on duration thresholds\n- Multiplication of the base rate by the number of intervals and the surcharge factor\nAll of this must be structured as a JSON expression tree with

In [54]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

{
    "synthesis_plan": "1. Start with the base rate for LAP at EAP: \u20ac3.50 per 15 minutes or part thereof.\n2. Convert the variable parking_duration_hours into number of 15-minute intervals (each interval counts as 1 unit).\n3. Apply a minimum charge of \u20ac3.20, but since the base rate is already \u20ac3.50, this is automatically satisfied.\n4. Since the query specifies 'no overnight parking', we do not subtract any free time.\n5. Check if the parking duration exceeds 48 or 72 hours, and apply surcharges accordingly:\n   - If parking_duration_hours \u2264 48: no surcharge\n   - If 48 < parking_duration_hours \u2264 72: surcharge of +100% (i.e., double the rate)\n   - If parking_duration_hours > 72: surcharge of +200% (i.e., triple the rate)\n6. The final charge is the base rate multiplied by the number of 15-minute intervals, adjusted by the applicable surcharge factor.\n7. Represent this logic using a conditional expression tree with branching on the value of parking_duration_

In [55]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

{
    "identified_constants_and_rules": [
        "The standard charge for Light Aircraft Parking (LAP) at East Aerodrome Parking (EAP) is \u20ac3.50 per 15 minutes or part thereof.",
        "Overnight parking from 2300-0600hrs is free of charge.",
        "Parking charges are calculated from the actual time of arrival (landed time) to the actual time of departure (airborne time) minus 30 minutes.",
        "For extended parking periods, surcharges apply: 48 hours up to 72 hours (including night-time) \u2192 Standard rate +100%; 72 hours and over \u2192 Standard rate +200%.",
        "However, since the query specifies 'no overnight parking', the overnight free period does not apply.",
        "The parking duration is a variable in hours, and the charge is per 15 minutes or part thereof.",
        "A minimum charge of \u20ac3.20 applies for light aircraft.",
        "Surcharges outlined in Table 1 apply for light aircraft parked in the designated parking zone and/or in any other EAP a

In [56]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

{
    "query_parameters": [
        {
            "name": "aircraft_stand_type",
            "status": "KNOWN",
            "value": "Light Aircraft Parking (LAP)"
        },
        {
            "name": "parking_duration_hours",
            "status": "SYMBOLIC",
            "value": null
        },
        {
            "name": "is_overnight_parking",
            "status": "KNOWN",
            "value": false
        },
        {
            "name": "sector",
            "status": "KNOWN",
            "value": "East Aerodrome Parking (EAP)"
        }
    ]
}


In [57]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "LESS_THAN",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 48,
                "description": "Threshold for first surcharge.",
                "unit": "HOURS"
            }
        },
        "if_true": {
            "type": "BINARY_OPERATION",
            "operator": "MULTIPLY",
            "left": {
                "type": "VALUE",
                "value": 3.5,
                "description": "Standard charge per 15 minutes for LAP at EAP.",
                "unit": "EUROS"
            },
            "right": {
                "type": "BINARY_OPERATI

In [ ]:
query = "Calculate the total parking charge for a Light Aircraft Parking (LAP) stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)
